# Tugas 2

**Buatlah model klasifikasi dengan menggunakan SVM berdasarkan data pada Percobaan 5 dengan menggunakan fitur histogram.**



# Langkah 1: Import Library yang Diperlukan

In [51]:
# Import Required Libraries
from pathlib import Path
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import numpy as np
from sklearn.metrics import accuracy_score
import os

In [52]:
# Image directories
train_dir = "images/training"
test_dir = "images/test"

# Langkah 2: Fungsi untuk Menghitung Histogram

In [53]:
def compute_histogram(image):
    # Convert the image to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    
    # Compute the histogram (16 bins for each channel)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [16, 16, 16], [0, 180, 0, 256, 0, 256])
    
    # Normalize the histogram
    cv2.normalize(hist, hist)
    
    # Flatten the histogram to create the feature vector
    return hist.flatten()


In [54]:
def standarized_input(image):
    # resize to w: 1100, h:600
    std_img = cv2.resize(image, (1100,600))

    return std_img

# Langkah 3: Fungsi untuk Membaca Gambar dan Label


In [55]:

def load_data(directory):
    data = []
    labels = []
    
    for label in ['day', 'night']:
        path = os.path.join(directory, label)
        for filename in os.listdir(path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                img_path = os.path.join(path, filename)
                
                # Read and resize the image
                image = cv2.imread(img_path)
                resized_image = standarized_input(image)
                
                # Compute histogram features
                hist = compute_histogram(resized_image)
                
                # Append features and label
                data.append(hist)
                labels.append(0 if label == 'night' else 1)  # 0 for night, 1 for day
                
    return np.array(data), np.array(labels)


# Langkah 4: Split Data train dan test

In [56]:
train_data, train_labels = load_data(train_dir)
test_data, test_labels = load_data(test_dir)

In [57]:
print(train_data.shape, train_labels[0])

(240, 4096) 1


# Langkah 5: Membuat Pipeline dengan PCA dan SVM

In [58]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

# Create a pipeline that combines PCA and the classifier
pipe = Pipeline(steps=[
    ('pca', PCA()),
    ('svm', SVC)
])

In [59]:
print(train_data[0])
print(train_labels[0])

type(train_labels)

[1.1891913e-05 5.5891991e-04 3.1156812e-03 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00]
1


numpy.ndarray

# Langkah 6: Melakukan Grid Search untuk Hyperparameter Tuning

In [60]:
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

# Create a pipeline with PCA and SVM
pipe = Pipeline(steps=[
    ('pca', PCA()),
    ('svm', svm.SVC())
])

# Define the parameter grid for GridSearch
param_grid = {
    'pca__n_components': [10, 20, 30, 50],  # Number of PCA components
    'svm__C': [0.1, 1, 10],                 # Regularization parameter for SVM
    'svm__kernel': ['linear', 'rbf']         # Kernel types
}

# Initialize GridSearchCV
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')

# Fit the model on training data
grid_search.fit(train_data, train_labels)
print(grid_search.best_params_)
print(grid_search.best_score_)


{'pca__n_components': 30, 'svm__C': 1, 'svm__kernel': 'rbf'}
0.9916666666666666


# Langkah 7: Evaluasi Model dan Hasil Terbaik

In [61]:
# Get the best model from GridSearch
best_model = grid_search.best_estimator_

# Predict on train data
train_predictions = best_model.predict(train_data)
# Predict on test data
test_predictions = best_model.predict(test_data)

# Evaluate accuracy
train_accuracy = accuracy_score(train_labels, train_predictions)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")

test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")



Train Accuracy: 99.17%
Test Accuracy: 96.88%


In [63]:
# Test dengan OpenCV

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    resized_frame = standarized_input(frame)
    hist = compute_histogram(resized_frame)
    pred = best_model.predict([hist])
    cv2.putText(frame, 'Day' if pred == 1 else 'Night', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()